In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
# from matplotlib import pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
ins_data = pd.read_csv('Insurance_Company_Complaints__Resolutions__Status__and_Recoveries.csv')

# To explore futher: 
* was the conclusion always 'justified' with a recovery? How many corrective actions were involved?
* Does this mean that an employee was negligent, at-fault, etc? Was the company itself reprimanded? 
* How many unfair claims practices were justified and what was their average recovery amount? 
* For chadwell: what is the most "alarming" reason for complaint for an insurer to receive, if it were to be justified?
* From a legal standpoint - would unfair claims practice come with the biggest possible lawsuit? Show Chadwell the list of Reasons and Subreasons
* Average length of time a case was open?

# After EDA/Visualization
* Generate a Hypothesis based on the data
* Use glassdoor sample data to test the hypothesis
* Perhaps load webscraped data into a SQL database? Mongo?
    * Glassdoor has an API - should I still webscrape to demonstrate this skill? 
* Look at the overall stats of reviews: 
    * Do the reviews follow a normal distribution? 
    * Are the results skewed towards positive or negative reviews?
    * Note this as a consideration in regards to whether the sample is i.i.d (are disgruntled employees posting more/less overall?)
* Are my findings similar to other studies that have looked at employee satisfaction vs customer satisfaction?

# Look at the published financials of the top 5 companies
* price per claims
* loss ratios
* Actuarily data
* Where are they making money, what do they tend to underwrite (what is their bread and butter service)
* It gets messy because a life insurance company can have different metrics than an auto company
* Might want to consider grouping by policy type
    * Anthem and ConnectiCare are two top Health Insurance companies

In [4]:
ins_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14509 entries, 0 to 14508
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      14509 non-null  object 
 1   File No.     14509 non-null  int64  
 2   Opened       14509 non-null  object 
 3   Closed       14078 non-null  object 
 4   Coverage     11915 non-null  object 
 5   SubCoverage  4879 non-null   object 
 6   Reason       14509 non-null  object 
 7   SubReason    14509 non-null  object 
 8   Disposition  14130 non-null  object 
 9   Conclusion   14130 non-null  object 
 10  Recovery     14509 non-null  float64
 11  Status       14509 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 1.3+ MB


In [52]:

# It looks like Insurance companies rarely pay out after a complaint, but when they do, the payout is very large
ins_data.describe()

,File No.,Recovery
count,14509.000000,14509.000000
mean,725908.174168,1331.318662
std,107689.450113,15838.502857
min,100831.000000,0.000000
25%,604805.000000,0.000000
50%,704227.000000,0.000000
75%,803691.000000,0.000000
max,902841.000000,843825.850000


In [40]:
# It appears that most NAN values are in Coverage or SubCoverage columns. Ask Chadwell if this is of interest, if not
# drop columns
ins_data[ins_data.isna().any(axis=1)]

,Company,File No.,Opened,Closed,Coverage,SubCoverage,Reason,SubReason,Disposition,Conclusion,Recovery,Status
1,21st Century Insurance Company,505848,12/31/2015,01/25/2016,Auto,NaN,Marketing/Sales,Producer Handling,Company Position Upheld,Unjustified,0.00,Closed
2,21st Century Insurance Company,601543,04/01/2016,08/12/2016,Auto,NaN,Underwriting,Cancellation,Insufficient Information,No Action Necessary,0.00,Closed
4,Allstate Fire and Casualty Insurance Company,803285,07/18/2018,08/06/2018,Auto,NaN,Premium and Rating,Premium Refund Delay,Corrective Action,Justified,95.59,Closed
5,National Life Insurance Company,902099,05/09/2019,NaN,Individual Life,Universal Life,Marketing/Sales,Producer Handling,NaN,NaN,0.00,Open
6,"Anthem Health Plans, Inc",901887,04/25/2019,05/13/2019,NaN,NaN,Underwriting,Cancellation,Coverage Granted,Justified,0.00,Closed
...,...,...,...,...,...,...,...,...,...,...,...,...
14498,"Anthem Health Plans, Inc",802680,06/08/2018,06/25/2018,NaN,NaN,Utilization Review,UR Procedure,Med Jurisdiction Explained,Furnished Information,0.00,Closed
14500,UnitedHealthcare Insurance Company,802854,06/20/2018,06/26/2018,NaN,NaN,Unfair Claims Practice,CPT Code Issue,Claim Settled,Furnished Information,159.36,Closed
14501,Bankers Life and Casualty Company,802634,06/06/2018,06/26/2018,NaN,NaN,Unfair Claims Practice,Claim Delays,Furnished Information,Furnished Information,0.00,Closed
14502,"Anthem Health Plans, Inc",802380,05/21/2018,06/26/2018,NaN,NaN,Unfair Claims Practice,Claim Procedure,Claim Settled,Furnished Information,804.50,Closed


In [8]:
ins_data.head()

,Company,File No.,Opened,Closed,Coverage,SubCoverage,Reason,SubReason,Disposition,Conclusion,Recovery,Status
0,Transamerica Life Insurance Company,803396,07/24/2018,08/06/2018,Credit A & H,No SubCov - AH,Unfair Claims Practice,Claim Delays,Company Position Upheld,Unjustified,0.00,Closed
1,21st Century Insurance Company,505848,12/31/2015,01/25/2016,Auto,NaN,Marketing/Sales,Producer Handling,Company Position Upheld,Unjustified,0.00,Closed
2,21st Century Insurance Company,601543,04/01/2016,08/12/2016,Auto,NaN,Underwriting,Cancellation,Insufficient Information,No Action Necessary,0.00,Closed
3,Prudential Insurance Company of America,805351,11/14/2018,12/04/2018,Individual Life,Whole Life,Marketing/Sales,Policy Service,Company Position Upheld,Unjustified,0.00,Closed
4,Allstate Fire and Casualty Insurance Company,803285,07/18/2018,08/06/2018,Auto,NaN,Premium and Rating,Premium Refund Delay,Corrective Action,Justified,95.59,Closed


In [179]:
company_complaints = ins_data.groupby('Company').count().sort_values('File No.', ascending=False)
# confirming correct code
# company_complaints = ins_data['Company'].value_counts()
company_complaints

,File No.,Opened,Closed,Coverage,SubCoverage,Reason,SubReason,Disposition,Conclusion,Recovery,Status
Company,,,,,,,,,,,
"Anthem Health Plans, Inc",1475,1475,1436,1161,1161,1475,1475,1444,1444,1475,1475
"ConnectiCare Insurance Company, Inc",598,598,582,456,456,598,598,584,584,598,598
Allstate Fire and Casualty Insurance Company,516,516,500,420,0,516,516,503,503,516,516
GEICO General Insurance Company,340,340,331,309,0,340,340,334,334,340,340
UnitedHealthcare Insurance Company,336,336,328,245,245,336,336,329,329,336,336
...,...,...,...,...,...,...,...,...,...,...,...
Federal Life Insurance Company,1,1,1,1,1,1,1,1,1,1,1
Executive Risk Specialty Insurance Company,1,1,1,1,0,1,1,1,1,1,1
Everest Reinsurance Company,1,1,1,1,0,1,1,1,1,1,1


## Looking at the contents of various columns

In [7]:
ins_data['Reason'].unique()

array(['Unfair Claims Practice', 'Marketing/Sales', 'Underwriting',
       'Premium and Rating', 'Utilization Review', 'FOI Inquiry',
       'Unknown', 'Employer Handling', 'Statute Violation', 'Other',
       'No Jurisdiction', 'Retro Review'], dtype=object)

In [9]:
ins_data['SubReason'].unique()

array(['Claim Delays', 'Producer Handling', 'Cancellation',
       'Policy Service', 'Premium Refund Delay',
       'Unsatisfactory Settlement', 'Premium/Rate Increase',
       'Denial of Claim', 'Premium/Notice', 'Medical Necessity',
       'Provider Contract Issue', 'Claim Procedure',
       'Unsatisfactory Offer', 'Non-Renewal', 'Actual Cash Value Dispute',
       'Eligibility of Provider', 'Contract Provision', 'No Subreason',
       'Policy Service Delay', 'Coordination of Benefit',
       'Misrepresentation', 'Excessive Charges', 'Loss of Use',
       'CPT Code Issue', 'Misleading Advertising',
       'Comparative Negligence', 'Failed to Remit Premium', 'Subrogation',
       'Diminished Value', 'Audit', 'UR Procedure', 'Refusal to Insure',
       'Other [Enter Sub-Reason]', 'Time Delay',
       'No Coverage/Premium Paid', 'Discontinuation & Replmnt',
       'Mandated Benefit', 'Case Management', 'Other Fees', 'Replacement',
       'Labor Rate', 'Delays', 'Usual and Customary Fees

In [10]:
ins_data['Conclusion'].unique()

array(['Unjustified', 'No Action Necessary', 'Justified', nan,
       'Questionable', 'Furnished Information', 'Voluntary Reconsider',
       'No Authority', 'Cross Reference Only'], dtype=object)

In [22]:
sub_reason = ins_data.groupby('SubReason').count()
sub_reason.sort_values('Company', ascending=False).head(20)

,Company,File No.,Opened,Closed,Coverage,SubCoverage,Reason,Disposition,Conclusion,Recovery,Status
SubReason,,,,,,,,,,,
Denial of Claim,2503,2503,2503,2396,2043,558,2503,2412,2412,2503,2503
Claim Delays,1935,1935,1935,1887,1545,227,1935,1891,1891,1935,1935
Claim Procedure,1061,1061,1061,1016,820,382,1061,1022,1022,1061,1061
Unsatisfactory Settlement,998,998,998,972,834,304,998,975,975,998,998
Excessive Charges,600,600,600,583,548,0,600,588,588,600,600
Premium/Notice,545,545,545,530,482,276,545,532,532,545,545
Premium/Rate Increase,540,540,540,520,472,472,540,522,522,540,540
Unsatisfactory Offer,523,523,523,510,432,0,523,512,512,523,523
Cancellation,435,435,435,424,380,93,435,424,424,435,435


In [27]:
# Looking at 'significantly sized' recovery amounts (I chose a number arbitrarily)
some_recovery = ins_data[ins_data['Recovery'] > 20000]
some_recovery.sort_values('Recovery', ascending=False)

,Company,File No.,Opened,Closed,Coverage,SubCoverage,Reason,SubReason,Disposition,Conclusion,Recovery,Status
3013,GEICO General Insurance Company,801471,03/26/2018,08/28/2018,Auto,NaN,Premium and Rating,Excessive Charges,Corrective Action,Justified,843825.85,Closed
11333,"Anthem Health Plans, Inc",601355,03/22/2016,03/30/2016,Individual Health,Air Ambulance,Utilization Review,Medical Necessity,Coverage Granted,Justified,838039.00,Closed
11121,"Anthem Health Plans, Inc",601355,03/22/2016,03/30/2016,NaN,NaN,Utilization Review,Medical Necessity,Coverage Granted,Justified,838039.00,Closed
12264,"Cigna HealthCare of Connecticut, Inc",405073,10/14/2014,04/29/2016,Group Health,Anesthesia,Unfair Claims Practice,Unsatisfactory Settlement,Claim Settled,Justified,404171.32,Closed
8927,State Farm Fire & Casualty Company,901019,03/01/2019,04/01/2019,Homeowners,NaN,Unfair Claims Practice,Claim Delays,Claim Settled,Justified,305000.00,Closed
...,...,...,...,...,...,...,...,...,...,...,...,...
2142,Carolina Casualty Insurance Company,600423,01/28/2016,03/17/2016,Worker's Compensation,NaN,Premium and Rating,Audit,Corrective Action,Justified,20608.00,Closed
10920,Underwriters at Lloyds London,702445,05/24/2017,07/18/2017,Travel Insurance,NaN,Unfair Claims Practice,Claim Procedure,Claim Settled,Furnished Information,20213.75,Closed
12801,Empire Fire and Marine Insurance Company,800695,02/09/2018,03/09/2018,Auto Liability,NaN,Unfair Claims Practice,Actual Cash Value Dispute,Corrective Action,Justified,20208.59,Closed
4201,"Anthem Health Plans, Inc",703224,07/17/2017,07/28/2017,Ind Medicare Supplemental,No SubCov - AH,Unfair Claims Practice,Claim Procedure,Claim Settled,Furnished Information,20085.09,Closed


In [32]:
# There were a surprising number of cases still open. Check to see when this dataset was last updated
ins_data.groupby('Status').count()
ins_data[ins_data['Status'] == 'Open']

,Company,File No.,Opened,Closed,Coverage,SubCoverage,Reason,SubReason,Disposition,Conclusion,Recovery,Status
5,National Life Insurance Company,902099,05/09/2019,NaN,Individual Life,Universal Life,Marketing/Sales,Producer Handling,NaN,NaN,0.0,Open
19,"Anthem Health Plans, Inc",902658,06/13/2019,NaN,NaN,NaN,Unfair Claims Practice,Provider Contract Issue,NaN,NaN,0.0,Open
20,Lincoln National Life Insurance Company (The),902672,06/14/2019,NaN,NaN,NaN,Unfair Claims Practice,Claim Procedure,NaN,NaN,0.0,Open
23,"ConnectiCare Benefits, Inc.",902657,06/13/2019,NaN,Exchange-Individual Hlth,High Deductible Hlth Plan,Premium and Rating,Premium/Notice,NaN,NaN,0.0,Open
88,National Life Insurance Company,902099,05/09/2019,NaN,NaN,NaN,Marketing/Sales,Producer Handling,NaN,NaN,0.0,Open
...,...,...,...,...,...,...,...,...,...,...,...,...
13904,New London County Mutual Insurance Company,902053,05/07/2019,NaN,Homeowners,NaN,Underwriting,Non-Renewal,NaN,NaN,0.0,Open
13919,Allstate Insurance Company,802282,05/14/2018,NaN,Homeowners,NaN,Premium and Rating,Excessive Charges,NaN,NaN,0.0,Open
13939,"ConnectiCare Benefits, Inc.",902112,05/09/2019,NaN,Exchange-Individual Hlth,Birth Control,Unfair Claims Practice,Denial of Claim,NaN,NaN,0.0,Open
14032,GEICO Casualty Company,902105,05/09/2019,NaN,Auto Physical Damage,NaN,Unfair Claims Practice,Denial of Claim,NaN,NaN,0.0,Open


In [36]:
# Disposition I assume is the company response. Counting those values and finding the top entries. 
# To explore: patterns with Refer-Judicial/Attorney
disposition_df = ins_data.groupby('Disposition').count().sort_values('File No.', ascending=False)
disposition_df.head(20)

,Company,File No.,Opened,Closed,Coverage,SubCoverage,Reason,SubReason,Conclusion,Recovery,Status
Disposition,,,,,,,,,,,
Company Position Upheld,3947,3947,3947,3931,3355,467,3947,3947,3947,3947,3947
Claim Settled,1736,1736,1736,1731,1374,547,1736,1736,1736,1736,1736
Furnished Information,1564,1564,1564,1561,1234,1170,1564,1564,1564,1564,1564
Contract Provision,1460,1460,1460,1452,1188,632,1460,1460,1460,1460,1460
Corrective Action,1340,1340,1340,1339,1100,349,1340,1340,1340,1340,1340
Refer-Judicial/Attorney,513,513,513,511,430,2,513,513,513,513,513
Rate Increase Explained,276,276,276,275,244,244,276,276,276,276,276
Premium Refund,259,259,259,259,221,107,259,259,259,259,259
Enter Arbitration,255,255,255,253,227,0,255,255,255,255,255


In [115]:
# Check the top companies to make sure they handle a similar number of claims each year - is the dataset "i.i.d",
# so to speak? 


# 623 distinct companies
ins_data['Company'].unique()

# Top 5 companies: Anthem Health Plans, ConnectiCare, Allstate, Geico, United
company_count = ins_data[['Company', 'File No.']].groupby('Company').count() \
                            .sort_values('File No.', ascending=False) \
                                .rename(columns={'File No.': 'Complaints Filed'})
# Checking to make sure the above command was correction. The sum of the counts == num rows. 
company_count['Complaints Filed'].sum()

# Ask Brian what a good normalizing number would be to compare the size of these companies and normalize for that.
# Revenue per year? Claims filed (if that number is even available)? Something from the stock market?
# Use Value at risk to normalize data
# Or, use something about expected payout? 

14509

# Exploring complaints that were referred to legal counsel

In [51]:
# Disposition - Refer/Judicial
# How many cases that were referred to Judicial were concluded in favor of the insured? 
# What was the average duration of these cases? What might this cost (find source for this)
# What was the most common reason/subreason for the complaint within this subgroup? 
# How many ended with recovery and what was the average payout? Higher or lower than without counsel? 
# For Chadwell - if the insured were to win a denial of claim vs comp neg case, which would likely have the higher
# payout, and by how much?
# Is a denial of claim likely just a small claims case? 


judicial_df = ins_data[ins_data['Disposition'] == 'Refer-Judicial/Attorney']
judicial_df.sort_values('Recovery', ascending = False)

,Company,File No.,Opened,Closed,Coverage,SubCoverage,Reason,SubReason,Disposition,Conclusion,Recovery,Status
9405,Allstate Fire and Casualty Insurance Company,702414,05/23/2017,08/03/2017,Auto Liability,NaN,Unfair Claims Practice,Denial of Claim,Refer-Judicial/Attorney,Questionable,2943.64,Closed
10288,Standard Fire Insurance Company (The),700304,01/19/2017,02/15/2017,Auto Liability,NaN,Unfair Claims Practice,Denial of Claim,Refer-Judicial/Attorney,Justified,2381.45,Closed
432,GEICO General Insurance Company,705754,12/21/2017,01/22/2018,Auto Liability,NaN,Unfair Claims Practice,Denial of Claim,Refer-Judicial/Attorney,Justified,2089.18,Closed
9732,Progressive Direct Insurance Company,603129,07/11/2016,08/22/2016,Auto Liability,NaN,Unfair Claims Practice,Denial of Claim,Refer-Judicial/Attorney,Justified,1500.00,Closed
9155,American Commerce Insurance Company,601942,04/26/2016,05/25/2016,Commercial Auto,NaN,Unfair Claims Practice,Denial of Claim,Refer-Judicial/Attorney,Justified,971.94,Closed
...,...,...,...,...,...,...,...,...,...,...,...,...
8885,Government Employees Insurance Company,602065,05/04/2016,05/23/2016,Auto Liability,NaN,Unfair Claims Practice,Denial of Claim,Refer-Judicial/Attorney,Unjustified,0.00,Closed
8882,Allstate Fire and Casualty Insurance Company,702629,06/06/2017,06/21/2017,Auto Liability,NaN,Unfair Claims Practice,Denial of Claim,Refer-Judicial/Attorney,Unjustified,0.00,Closed
8876,Progressive Casualty Insurance Company,600713,02/17/2016,03/21/2016,Auto Liability,NaN,Unfair Claims Practice,Labor Rate,Refer-Judicial/Attorney,Unjustified,0.00,Closed
8867,IDS Property Casualty Insurance Company,605525,12/05/2016,01/04/2017,Auto Liability,NaN,Unfair Claims Practice,Comparative Negligence,Refer-Judicial/Attorney,Unjustified,0.00,Closed


In [122]:
# Which have the most justified complaints?
y = ins_data[ins_data['Conclusion'] == 'Justified'].reset_index()

y[['Company', 'Conclusion']].groupby('Company').count().sort_values('Conclusion', ascending=False)

,Conclusion
Company,
"Anthem Health Plans, Inc",198
UnitedHealthcare Insurance Company,74
Allstate Fire and Casualty Insurance Company,62
LM General Insurance Company,57
"ConnectiCare Insurance Company, Inc",49
...,...
Narragansett Bay Insurance Company,1
Minnesota Life Insurance Company,1
American Zurich Insurance Company,1


In [105]:
# which have the most corrective action? (There are 623 companies)
# add column that shows the recovery amount sum per company for CA
z = ins_data[['Company', 'Disposition']]
corrective_act_count = z[z['Disposition'] == 'Corrective Action'].groupby('Company').count() \
                        .sort_values(by='Disposition', ascending=False).reset_index()
corrective_act_count = corrective_act_count.rename(columns={'Disposition': 'Corrective Action Count'})
corrective_act_count

,Company,Corrective Action Count
0,"Anthem Health Plans, Inc",59
1,Allstate Fire and Casualty Insurance Company,46
2,LM General Insurance Company,38
3,GEICO General Insurance Company,33
4,State Farm Mutual Automobile Insurance Company,29
...,...,...
254,"Paratransit Insurance Company, A Mutual Risk R...",1
255,Genworth Life Insurance Company,1
256,Genworth Life and Annuity Insurance Company,1
257,Old Republic Insurance Company,1


In [106]:
# Insert column that shows the ratio of CA Counts / total complaints filed

corrective_act_count = 


# Grouping by policy type
- I don't want to aggregate over the column, but instead just look at data in coverage type buckets
- Perhaps groupby is not the best choice
- There are 69 unique coverage types
- Can any of these coverage types be collapsed? (Auto Liability, Auto, Auto Physical Damage, for example)
- SubCoverage had relatively few entries

In [136]:
ins_data['Coverage'].nunique()
ins_data[['Coverage', 'Company']].groupby('Coverage').count().sort_values('Company', ascending=False).head(20)

,Company
Coverage,
Auto Liability,2335
Homeowners,1610
Group Health,1442
Auto Physical Damage,915
Auto,852
Individual Health,738
Exchange-Individual Hlth,600
Individual Life,556
Individual Long Term Care,430


In [162]:
# Exploring all 'Auto' related coverage types - need to drop NAN first
auto_df = ins_data[ins_data['Coverage'].notna()]
auto_df = auto_df[auto_df['Coverage'].str.contains('Auto')]
auto_df

,Company,File No.,Opened,Closed,Coverage,SubCoverage,Reason,SubReason,Disposition,Conclusion,Recovery,Status
1,21st Century Insurance Company,505848,12/31/2015,01/25/2016,Auto,NaN,Marketing/Sales,Producer Handling,Company Position Upheld,Unjustified,0.00,Closed
2,21st Century Insurance Company,601543,04/01/2016,08/12/2016,Auto,NaN,Underwriting,Cancellation,Insufficient Information,No Action Necessary,0.00,Closed
4,Allstate Fire and Casualty Insurance Company,803285,07/18/2018,08/06/2018,Auto,NaN,Premium and Rating,Premium Refund Delay,Corrective Action,Justified,95.59,Closed
7,Standard Fire Insurance Company (The),803122,07/06/2018,08/06/2018,Auto,NaN,Marketing/Sales,Producer Handling,Corrective Action,Questionable,0.00,Closed
14,Progressive Casualty Insurance Company,805221,11/05/2018,12/04/2018,Auto Liability,NaN,Unfair Claims Practice,Denial of Claim,Company Position Upheld,Unjustified,0.00,Closed
...,...,...,...,...,...,...,...,...,...,...,...,...
14451,Progressive Direct Insurance Company,802642,06/06/2018,06/20/2018,Auto Liability,NaN,Marketing/Sales,No Coverage/Premium Paid,Company Position Upheld,Unjustified,0.00,Closed
14452,Old Dominion Insurance Company,802222,05/09/2018,06/20/2018,Auto,NaN,Premium and Rating,Premium/Notice,Company Position Upheld,Unjustified,0.00,Closed
14458,SAFECO Insurance Company of Illinois,802593,06/05/2018,06/20/2018,Auto Physical Damage,NaN,Unfair Claims Practice,Actual Cash Value Dispute,Corrective Action,Justified,0.00,Closed
14462,LM General Insurance Company,802498,05/29/2018,06/20/2018,Auto Physical Damage,NaN,Unfair Claims Practice,Labor Rate,Company Position Upheld,Unjustified,0.00,Closed


In [186]:
auto_df['Company'].value_counts()

Allstate Fire and Casualty Insurance Company    412
GEICO General Insurance Company                 305
GEICO Indemnity Company                         257
Progressive Direct Insurance Company            232
Progressive Casualty Insurance Company          210
                                               ... 
Allied Insurance Company of America               1
Dealers Assurance Company                         1
Valley Forge Insurance Company                    1
Navigators Insurance Company                      1
Atlantic Mutual Insurance Company                 1
Name: Company, Length: 263, dtype: int64

,Company,File No.,Opened,Closed,Coverage,SubCoverage,SubReason,Disposition,Conclusion,Recovery,Status
Reason,,,,,,,,,,,
FOI Inquiry,9,9,9,8,9,0,9,8,8,9,9
Marketing/Sales,90,90,90,89,90,0,90,89,89,90,90
No Jurisdiction,2,2,2,2,2,0,2,2,2,2,2
Other,1,1,1,1,1,0,1,1,1,1,1
Premium and Rating,552,552,552,543,552,0,552,545,545,552,552
Underwriting,239,239,239,233,239,0,239,234,234,239,239
Unfair Claims Practice,3449,3449,3449,3368,3449,0,3449,3381,3381,3449,3449
Unknown,3,3,3,3,3,0,3,3,3,3,3


In [185]:
# Within auto complaints, Unfair claims practice is attributable to the the largest total recovery amount
auto_df.groupby('Reason').sum().sort_values('Recovery', ascending=False)

,File No.,Recovery
Reason,,
Unfair Claims Practice,2509941757,1694697.05
Premium and Rating,401273311,903654.19
Underwriting,168927109,22125.86
Marketing/Sales,63177039,14592.92
FOI Inquiry,6016428,0.00
No Jurisdiction,1706788,0.00
Other,500210,0.00
Unknown,2112589,0.00


# Exploring time

In [187]:
ins_data.head(10)

,Company,File No.,Opened,Closed,Coverage,SubCoverage,Reason,SubReason,Disposition,Conclusion,Recovery,Status
0,Transamerica Life Insurance Company,803396,07/24/2018,08/06/2018,Credit A & H,No SubCov - AH,Unfair Claims Practice,Claim Delays,Company Position Upheld,Unjustified,0.00,Closed
1,21st Century Insurance Company,505848,12/31/2015,01/25/2016,Auto,NaN,Marketing/Sales,Producer Handling,Company Position Upheld,Unjustified,0.00,Closed
2,21st Century Insurance Company,601543,04/01/2016,08/12/2016,Auto,NaN,Underwriting,Cancellation,Insufficient Information,No Action Necessary,0.00,Closed
3,Prudential Insurance Company of America,805351,11/14/2018,12/04/2018,Individual Life,Whole Life,Marketing/Sales,Policy Service,Company Position Upheld,Unjustified,0.00,Closed
4,Allstate Fire and Casualty Insurance Company,803285,07/18/2018,08/06/2018,Auto,NaN,Premium and Rating,Premium Refund Delay,Corrective Action,Justified,95.59,Closed
5,National Life Insurance Company,902099,05/09/2019,NaN,Individual Life,Universal Life,Marketing/Sales,Producer Handling,NaN,NaN,0.00,Open
6,"Anthem Health Plans, Inc",901887,04/25/2019,05/13/2019,NaN,NaN,Underwriting,Cancellation,Coverage Granted,Justified,0.00,Closed
7,Standard Fire Insurance Company (The),803122,07/06/2018,08/06/2018,Auto,NaN,Marketing/Sales,Producer Handling,Corrective Action,Questionable,0.00,Closed
8,Liberty Insurance Corporation,803184,07/11/2018,08/06/2018,NaN,NaN,Premium and Rating,Premium Refund Delay,Corrective Action,Justified,924.00,Closed
9,"Anthem Health Plans, Inc",802871,06/21/2018,08/06/2018,Group Health,Emergency,Unfair Claims Practice,Unsatisfactory Settlement,Contract Provision,Furnished Information,0.00,Closed


In [201]:
# Inspecting an element from the opened column shows the dates are cast as strings. 
dt_df = ins_data[ins_data['Opened'].notna()]
type(dt_df['Closed'][0])

dt_df['Closed']= pd.to_datetime(dt_df['Closed'])
dt_df['Opened']= pd.to_datetime(dt_df['Opened'])



In [203]:
dt_df['Claim Duration'] = dt_df['Closed'] - dt_df['Opened']

In [211]:
# Why would there be two file numbers? 
dt_df[dt_df['Recovery'] > 2000].sort_values('Claim Duration', ascending = False).head(10)

,Company,File No.,Opened,Closed,Coverage,SubCoverage,Reason,SubReason,Disposition,Conclusion,Recovery,Status,Claim Duration
8135,"Anthem Health Plans, Inc",404944,2014-10-02,2017-04-26,NaN,NaN,Unfair Claims Practice,Claim Delays,Claim Settled,Justified,75844.76,Closed,937 days
10064,"Anthem Health Plans, Inc",404944,2014-10-02,2017-04-26,Exchange-Individual Hlth,Autism,Unfair Claims Practice,Claim Delays,Claim Settled,Justified,75844.76,Closed,937 days
9969,"Anthem Health Plans, Inc",500440,2015-02-03,2016-11-09,Exchange-Individual Hlth,Out-Patient,Unfair Claims Practice,CPT Code Issue,Claim Paid With Interest,Justified,49924.93,Closed,645 days
12264,"Cigna HealthCare of Connecticut, Inc",405073,2014-10-14,2016-04-29,Group Health,Anesthesia,Unfair Claims Practice,Unsatisfactory Settlement,Claim Settled,Justified,404171.32,Closed,563 days
14153,"Anthem Health Plans, Inc",605709,2016-12-15,2018-06-01,Exchange-Individual Hlth,Ambulance,Unfair Claims Practice,Provider Contract Issue,Claim Settled,Furnished Information,32169.00,Closed,533 days
6566,"Anthem Health Plans, Inc",801113,2018-03-06,2019-03-18,Group Health,No SubCov - AH,Unfair Claims Practice,Unsatisfactory Settlement,Claim Settled,Justified,28429.59,Closed,377 days
14212,Quincy Mutual Fire Insurance Company,703461,2017-07-31,2018-07-31,Homeowners,NaN,Unfair Claims Practice,Unsatisfactory Settlement,Additional Money Received,Unjustified,2366.76,Closed,365 days
14278,Transamerica Life Insurance Company,702689,2017-06-09,2018-06-08,Individual Long Term Care,Partnership,Unfair Claims Practice,Claim Delays,Furnished Information,Furnished Information,93090.99,Closed,364 days
14264,Transamerica Life Insurance Company,702689,2017-06-09,2018-06-08,NaN,NaN,Unfair Claims Practice,Claim Delays,Furnished Information,Furnished Information,93090.99,Closed,364 days
2333,"ConnectiCare Insurance Company, Inc",603946,2016-08-23,2017-07-21,Group Health,Lab Services,Unfair Claims Practice,Claim Procedure,Claim Settled,Voluntary Reconsider,3950.06,Closed,332 days
